In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

In [3]:
data = pd.read_csv('sign_mnist_train.csv')
data

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27450,13,189,189,190,190,192,193,193,193,193,...,132,165,99,77,52,200,234,200,222,225
27451,23,151,154,157,158,160,161,163,164,166,...,198,198,198,198,198,196,195,195,195,194
27452,18,174,174,174,174,174,175,175,174,173,...,121,196,209,208,206,204,203,202,200,200
27453,17,177,181,184,185,187,189,190,191,191,...,119,56,27,58,102,79,47,64,87,93


In [4]:
data.isna().any().sum()

0

In [5]:
y_train = data['label']
X_train = data.drop(columns = 'label')

In [6]:
# For testing data
data2= pd.read_csv('sign_mnist_test.csv')
data2

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,6,149,149,150,150,150,151,151,150,151,...,138,148,127,89,82,96,106,112,120,107
1,5,126,128,131,132,133,134,135,135,136,...,47,104,194,183,186,184,184,184,182,180
2,10,85,88,92,96,105,123,135,143,147,...,68,166,242,227,230,227,226,225,224,222
3,0,203,205,207,206,207,209,210,209,210,...,154,248,247,248,253,236,230,240,253,255
4,3,188,191,193,195,199,201,202,203,203,...,26,40,64,48,29,46,49,46,46,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7167,1,135,119,108,102,105,99,61,103,121,...,108,112,116,114,118,180,184,176,167,163
7168,12,157,159,161,164,166,166,171,174,175,...,213,213,213,214,213,211,210,210,209,208
7169,2,190,191,190,191,190,190,192,192,191,...,216,215,213,214,214,213,210,211,209,208
7170,4,201,205,208,209,214,216,218,223,226,...,112,169,255,255,237,113,91,67,70,63


In [7]:
y_test = data2['label']
X_test = data2.drop(columns = 'label')

In [19]:
# Testing reshape
test_X_train = np.zeros((27455, 28, 28))

for i in range(27455):
    test_X_train[i] = X_train.loc[i].to_numpy().reshape(28, 28)


array([[107., 118., 127., 134., 139., 143., 146., 150., 153., 156., 158.,
        160., 163., 165., 159., 166., 168., 170., 170., 171., 171., 171.,
        172., 171., 171., 170., 170., 169.],
       [111., 121., 129., 135., 141., 144., 148., 151., 154., 157., 160.,
        163., 164., 170., 119., 152., 171., 171., 170., 171., 172., 172.,
        172., 172., 172., 171., 171., 170.],
       [113., 123., 131., 137., 142., 145., 150., 152., 155., 158., 161.,
        163., 164., 172., 105., 142., 170., 171., 171., 171., 172., 172.,
        173., 173., 172., 171., 171., 171.],
       [116., 125., 133., 139., 143., 146., 151., 153., 156., 159., 162.,
        163., 167., 167.,  95., 144., 171., 172., 172., 172., 172., 172.,
        173., 173., 173., 172., 172., 171.],
       [117., 126., 134., 140., 145., 149., 153., 156., 158., 161., 163.,
        164., 175., 156.,  87., 154., 172., 173., 173., 173., 173., 173.,
        174., 174., 174., 173., 172., 172.],
       [119., 128., 136., 142., 146

In [22]:
from sklearn.model_selection import cross_val_score, RepeatedKFold, GridSearchCV
from sklearn.svm import SVC

model = SVC(random_state=42, C=1, kernel='linear').fit(X_train, y_train)

In [23]:
model.score(X_test, y_test)

0.7816508644729504

In [ ]:
from sklearn.model_selection import cross_val_score, RepeatedKFold, GridSearchCV
from sklearn.svm import SVC

model = SVC(random_state=42)
p_grid = {'C': [0.01, 0.5, 10]}

rkf = RepeatedKFold(n_splits=5, n_repeats=1, random_state=42)
clf1 = GridSearchCV(estimator=model, param_grid=p_grid, cv=rkf, verbose=3)
clf1.fit(X_train, y_train)

# View the Grid Search CV Results
print(clf1.cv_results_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
